### 데이터 값 목록 전체

* 데이터 값 전체 자료를 csv 파일로 저장

In [2]:
import os
import pandas as pd

# 파일 저장 위치
path_dir = "D:/PythonProject/data-gatherer/dart_fs_notes/items/"
file_list = os.listdir(path_dir)

df_all = pd.DataFrame()    
for file_name in file_list:
    if file_name != 'items_all.csv' and not file_name.startswith('__'):
        df = pd.read_csv(path_dir + file_name, dtype=object)
        df_all = pd.concat([df_all, df])
    
df_all.drop_duplicates(subset=['corp_code','rpt_num'], keep='first', inplace=True, ignore_index=True)            
df_all.to_csv(path_dir + 'items_all.csv', index=False)

### 파일에 저장된 데이터 값 목록 검색

In [3]:
import pandas as pd
# 데이터 Loading
path_dir = "D:/PythonProject/data-gatherer/dart_fs_notes/items/"
df = pd.read_csv(path_dir + 'items_all.csv', dtype=object)
num_of_rpt = int(df.shape[0])
num_of_corp = len(df['corp_code'].unique())
print(f"기업 {num_of_corp}, 사업보고서 {num_of_rpt}")

기업 342, 사업보고서 2444


In [4]:
df['doc_result'].value_counts()

doc_result
OK                              2367
No Financial Statement Notes      77
Name: count, dtype: int64

In [5]:
df['unit'].value_counts()

unit
천원     1643
백만원     323
원       180
Name: count, dtype: int64

In [10]:
import sys
module_path = "D:\PythonProject\data-gatherer\dart_fs_notes"
sys.path.append(module_path)
import myutil

In [21]:
result_chk = df[['doc_result', 'item_result']].value_counts()
df_result_chk = pd.DataFrame(result_chk)
df_result_chk.reset_index(inplace=True)
df_result_chk.to_csv("D:/PythonProject/data-gatherer/dart_fs_notes/output/" + "result_chk_" + myutil.now_dt_str() + ".csv", index=False)

In [22]:
# 수기 보정 필요한 건
str_expr = "val_a == '-1.0' or val_b == '-1.0'"
df_except = df.query(str_expr)
df_except.to_csv("D:/PythonProject/data-gatherer/dart_fs_notes/output/" + "except_" + myutil.now_dt_str() + ".csv", index=False)

In [23]:
# 오류 찾기: "금액 단위 찾기 오류" ~ 재무제표 주석 문서가 있으면서 단위는 없고 금액은 있는 것
df_err_chk1 = df.query("doc_result == 'OK' and unit.isna() and (val_a != '0.0' or val_b != '0.0')")
df_err_chk1.to_csv("D:/PythonProject/data-gatherer/dart_fs_notes/output/" + "err_chk1_" + myutil.now_dt_str() + ".csv", index=False)

In [24]:
# 오류 찾기: "값 찾기 오류" ~ 재무제표 주석 문서가 있으면서 val_a는 못 찾고 val_b는 찾은 경우
df_err_chk2 = df.query("doc_result == 'OK' and val_a == '0.0' and val_b != '0.0'")
df_err_chk2.to_csv("D:/PythonProject/data-gatherer/dart_fs_notes/output/" + "err_chk2_" + myutil.now_dt_str() + ".csv", index=False)

In [25]:
# 오류 찾기: 재무제표 주석 문서가 있으면서 금액과 단위가 없는 것 ~ 오류인지 아닌지 직접 확인 필요
df_err_chk3 = df.query("doc_result == 'OK' and unit.isna() and val_a == '0.0' and val_b == '0.0'")
df_err_chk3.to_csv("D:/PythonProject/data-gatherer/dart_fs_notes/output/" + "err_chk3_" + myutil.now_dt_str() + ".csv", index=False)